# Capstone project notebook

## <span style="color:red">1. _Get the data_</span>

### Import the packages needed

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np # library to handle data in a vectorized manner

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#uncomment conda installation if needed (i.e. if geocoder is not installed)
#Doesn't work
#!conda install -c conda-forge geocoder 
#import geocoder
#Doesn't work either
#!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Import postal codes data from wikipedia URL

In [2]:
html_data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content
soup = BeautifulSoup(html_data, 'html.parser')

### Create a dataframe from the first table in the page

In [3]:
table=soup('table')[0]
postal_codes=pd.read_html(str(table))[0]
postal_codes.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Cleanup the data

In [4]:
#Remove entries without borough
postal_codes=postal_codes[postal_codes.Borough != 'Not assigned']
postal_codes.reset_index(drop=True, inplace=True)
#Copy borough to neighborhood, where missing
postal_codes.Neighbourhood = postal_codes.apply(lambda x: x.Borough if x.Neighbourhood=='Not assigned' else x.Neighbourhood, axis=1 )
#Keep only distinct postal codes / borough
postcodes = pd.DataFrame(postal_codes.groupby(['Postcode','Borough']).count().reset_index())
#Assign a list of Neighborhood to each postcode
postcodes['Neighbourhood'] = postcodes.apply(lambda row: ", ".join(map(str,(list(postal_codes[postal_codes['Postcode']==row['Postcode']]["Neighbourhood"])))), axis=1)
postcodes.rename(columns={"Postcode": "Postalcode"}, inplace=True)
postcodes.head()


,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Show the shape of the resulting dataframe

In [5]:
postcodes.shape

(103, 3)

## <span style="color:red">2. _Get the coordinates_</span>

### Define the function that will get the data

In [6]:
gsd=pd.read_csv("http://cocl.us/Geospatial_data")
gsd.set_index('Postal Code', inplace=True)

#Create the function to get the coordinates
def getTorontoCoords(pcode):
    # initialize your variable to None
    #lat_lng_coords = None
    #print("Getting coordinates for "+pcode+".")

    #Get from CSV. If geocodings are broken
    try:
        loc=gsd.loc[pcode,['Latitude','Longitude']]
        return (loc[0], loc[1])
    except:
        print("Can't get coordinates for "+pcode)
        return (None, None)

    #address = '{}, Toronto, Canada'.format(pcode)
    #geolocator = Nominatim(user_agent="to_explorer")
    #loc=geolocator.geocode(address)
    # loop until you get the coordinates
    #while(lat_lng_coords is None):
    #  g = geocoder.google('{}, Toronto'.format(pcode))
    #  lat_lng_coords = g.latlng
    #  if (lat_lng_coords is None):
    #    print("Can't get coordinates for "+pcode+". Retrying...")
    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]

    #if loc:
    #    return (loc.latitude, loc.longitude)
    #else:
    #    return (None, None)
    

### Set all the coordinates for each postcode

In [7]:

postcodes['latlong'] = postcodes.apply(lambda row: getTorontoCoords(row['Postalcode']), axis=1)
#Separate the coordinates and remove the combination
postcodes['Latitude'] = postcodes.apply(lambda row:row['latlong'][0], axis=1)
postcodes['Longitude'] = postcodes.apply(lambda row:row['latlong'][1], axis=1)
postcodes.drop(['latlong'], axis=1, inplace=True)
postcodes.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## <span style="color:red">3. Clustering Neighborhoods</span>

### Get coordinates for the M5G postal code (just to get a starting point...)

In [8]:
neighborhood_latitude=gsd.loc['M5G',['Latitude']][0]
neighborhood_longitude=gsd.loc['M5G',['Longitude']][0]
print(neighborhood_latitude,neighborhood_longitude)

43.6579524 -79.3873826


### Create map of Toronto using latitude and longitude values

In [9]:
map_toronto = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=11)

# add markers to map
for pc, lat, lng, borough, neighbourhood in zip(postcodes['Postalcode'], postcodes['Latitude'], postcodes['Longitude'], postcodes['Borough'], postcodes['Neighbourhood']):
    label = '{} {}, {}'.format(pc, neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare credentials and version

In [11]:
# @hidden_cell
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


### Get the venues

In [12]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d68ed30446ea6002c777a6d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 85,
  'suggestedBounds': {'ne': {'lat': 43.6624524045, 'lng': -79.38117421839567},
   'sw': {'lat': 43.6534523955, 'lng': -79.39359098160432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '537d4d6d498ec171ba22e7fe',
       'name': "Jimmy's Coffee",
       'location': {'address': '82 Gerrard Street W',
        'crossStreet': 'Gerrard & LaPlante',
        'lat': 43.65842123574496,
        'lng': -79.38561319551111,
        'label

### Function to get the nearby venues for a certain coordinate

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Create a dataframe with postalcode as a column

In [14]:
# Use this to analyze just a subset
#downtown_data = postcodes[postcodes['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
#Use this to analyze all of the available data
toronto_data = postcodes.reset_index(drop=True)
toronto_data.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Get Toronto venues

In [15]:
print("getting data...")
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
print("done")

getting data...
done


### Remove Neighborhoods categories because they're just confusing

In [16]:
print("Before: "+str(toronto_venues.shape))
toronto_venues=toronto_venues[toronto_venues["Venue Category"]!='Neighborhood']
print("After: "+str(toronto_venues.shape))
toronto_venues.groupby('Neighborhood').count()

Before: (2258, 7)
After: (2254, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",99,99,99,99,99,99
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,57,57,57,57,57,57


### Analyze neighbourhoods

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Get the mean of each venue category groupe by neighborhood

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,...,0.0,0.020202,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.0,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


### Show the top categories for each neighborhood

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.05
2             Bar  0.04
3      Steakhouse  0.04
4  Cosmetics Shop  0.03


----Agincourt----
               venue  freq
0     Breakfast Spot  0.25
1             Lounge  0.25
2     Sandwich Place  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground   0.5
1                Park   0.5
2  Miscellaneous Shop   0.0
3       Movie Theater   0.0
4               Motel   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.22
1           Beer Store  0.11
2  Fried Chicken Joint  0.11
3       Sandwich Place  0.11
4          Coffee Shop  0.11


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1             Gym  0.11
2    Sk

### Define a function to get a list of the top most common venues

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,American Restaurant
1,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Skating Rink,Yoga Studio
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Eastern European Restaurant,Discount Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Pizza Place,Coffee Shop
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Coffee Shop,Pub,Dance Studio


### Cluster neighborhoods

In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, max_iter=500, n_init=15).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
#Remove rows without data
toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)
toronto_merged.head() 

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Moving Target,Bar,Eastern European Restaurant,Dog Run,Doner Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Pizza Place,Spa,Intersection,Rental Car Location,Breakfast Spot
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant


In [25]:
# create map
map_clusters = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(int(cluster)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the produced clusters

#### Cluster 1

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Scarborough Village,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run
14,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Eastern European Restaurant,Discount Store


#### Cluster 2

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
94,"Cloverdale, Islington, Martin Grove, Princess ...",Bank,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop


#### Cluster 3

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Rouge, Malvern",Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run
1,"Highland Creek, Rouge Hill, Port Union",Moving Target,Bar,Eastern European Restaurant,Dog Run,Doner Restaurant
2,"Guildwood, Morningside, West Hill",Pizza Place,Spa,Intersection,Rental Car Location,Breakfast Spot
3,Woburn,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Doner Restaurant
4,Cedarbrae,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant
6,"East Birchmount Park, Ionview, Kennedy Park",Playground,Bus Station,Department Store,Coffee Shop,Yoga Studio
7,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Intersection,Bus Station
8,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Diner,Dog Run,Doner Restaurant
9,"Birch Cliff, Cliffside West",Café,General Entertainment,Skating Rink,College Stadium,Concert Hall
10,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Furniture / Home Store,Chinese Restaurant,Pet Store,Vietnamese Restaurant


#### Cluster 4

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
97,"Emery, Humberlea",Baseball Field,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop


#### Cluster 5

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,York Mills West,Electronics Store,Bank,Park,Convenience Store,Yoga Studio
25,Parkwoods,Food & Drink Shop,Park,Pool,Yoga Studio,Discount Store
30,"CFB Toronto, Downsview East",Electronics Store,Airport,Park,Yoga Studio,Dog Run
40,East Toronto,Coffee Shop,Park,Convenience Store,Yoga Studio,Eastern European Restaurant
44,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Dumpling Restaurant
50,Rosedale,Park,Trail,Playground,Building,Yoga Studio
64,"Forest Hill North, Forest Hill West",Bus Line,Park,Sushi Restaurant,Jewelry Store,Trail
74,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Construction & Landscaping
79,"Downsview, North Park, Upwood Park",Basketball Court,Bakery,Construction & Landscaping,Park,Yoga Studio
90,"The Kingsway, Montgomery Road, Old Mill North",River,Park,Yoga Studio,Dumpling Restaurant,Discount Store
